In [3]:
import os

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\akuma\\Desktop\\CV\\end-to-end-cv-project-objectDetection'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
  root_dir: Path
  source_url: str
  local_data_file: Path
  unzip_dir: Path

In [11]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directories


In [17]:
class ConfigurationManager:
  def __init__(
    self,
    config_filepath = CONFIG_FILE_PATH,
    params_filepath = PARAMS_FILE_PATH,
    
  ):
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    
    create_directories([self.config.artifacts_root])
  
  def get_data_ingestion_config(self)->DataIngestionConfig:
    config = self.config.data_ingestion
    create_directories([config.root_dir])
    data_ingestion_config = DataIngestionConfig(
      root_dir=config.root_dir,
      source_url=config.source_url,
      local_data_file=config.local_data_file,
      unzip_dir=config.unzip_dir
    )
    return data_ingestion_config

In [14]:
import os 
import zipfile
import gdown
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [16]:
class DataIngestion:
  def __init__(self,config:DataIngestionConfig):
    self.config = config
  
  def download_file(self)->str:
    try:
      dataset_url = self.config.source_url
      zip_download_dir = self.config.local_data_file
      os.makedirs(f"{self.config.root_dir}",exist_ok=True)
      logger.info(f"Downloading data from {dataset_url} into file: {zip_download_dir}")
      
      file_id = dataset_url.split("/")[-2]
      prefix = 'https://drive.google.com/uc?/export=download&id='
      drive_link = prefix + file_id
      gdown.download(drive_link,zip_download_dir)
      
      logger.info(f"Downloaded data from {dataset_url} into file: {zip_download_dir}")
    except Exception as error:
      raise error
    
  def extract_zip_file(self):
    unzip_path = self.config.unzip_dir
    os.makedirs(unzip_path,exist_ok=True)
    with zipfile.ZipFile(self.config.local_data_file,"r") as reader:
      reader.extractall(unzip_path)
      

In [18]:
try:
  config = ConfigurationManager() ###configuration manager
  data_ingestion_config = config.get_data_ingestion_config() #extracting the config using this config
  data_ingestion = DataIngestion(config=data_ingestion_config) #using the config into the function
  
  data_ingestion.download_file()
  data_ingestion.extract_zip_file()
except Exception as error:
  raise error

[2024-09-10 01:52:09,261: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-10 01:52:09,280: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-10 01:52:09,284: INFO: common: Created directory at: artifacts]
[2024-09-10 01:52:09,288: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-09-10 01:52:09,292: INFO: 4070247016: Downloading data from https://drive.google.com/file/d/1bho-vzvxjymNMgREyQrHiEe8Z39cFsl2/view?usp=sharing into file: artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1bho-vzvxjymNMgREyQrHiEe8Z39cFsl2
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1bho-vzvxjymNMgREyQrHiEe8Z39cFsl2&confirm=t&uuid=aa143cb5-9350-41f0-a823-f47289ab1eca
To: c:\Users\akuma\Desktop\CV\end-to-end-cv-project-objectDetection\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:04<00:00, 11.5MB/s]

[2024-09-10 01:52:19,163: INFO: 4070247016: Downloaded data from https://drive.google.com/file/d/1bho-vzvxjymNMgREyQrHiEe8Z39cFsl2/view?usp=sharing into file: artifacts/data_ingestion/data.zip]
